In [1]:
import open3d as o3d
import numpy as np

import os
import json
import logging

In [2]:
def write_labelFile(label, labelFile):
    """Function to write label dict to json format file.
    
    Args:
        label(dict)   : The dict of label.
        labelFile(str): The label file, json format.
        
    Returns:
        True or False
    """
    _flag = True
    try:
        with open(labelFile, 'w') as _labelfile:
            json.dump(label, _labelfile, indent=2, sort_keys=True)
    except TypeError:
        _flag = False
        raise Exception("[Error]: Unable to serialize it to json: %s" % label)
    
    return _flag
    

def load_pcd(filename):
    """Load point cloud file.
    
    Input a pcd file, it will return open3d format pointcloud.
    
    Args:
        filename(str): pcd file path.
        
    Returns:
        open3d.geometry.PointCloud
    """
    if os.path.exists(filename):
        return o3d.io.read_point_cloud(filename)
    else:
        logging.warning('[Warnning]: %s is not found.' % filename)
        

def label3D(pcd):
    """Label point cloud file by a point with label.
    
    Press [SHIFT + CLICK_LEFT] to select a point, it will show
    a tiny dialog which got the label name you input.
    
    Args:
        pcd(open3d.geometry.PointCloud): open3d.geometry.PointCloud.
        label_file
    
    Returns:
        The points coordinates: {'labelname': {'x': 0, 'y': 0, 'z': 0}, ...}
    """
    _visEditing = o3d.visualization.VisualizerWithEditing()
    _visEditing.create_window()
    _visEditing.add_geometry(pcd)
    _visEditing.run()
    _visEditing.destroy_window()
    _picked_points = _visEditing.get_picked_points()
    _picked_map = _visEditing.get_picked_map()
    for k, v in _picked_map.items():
        # If you delete some points, remove from _packed_map.
        if _picked_map[k] not in _picked_points:
            _picked_map.pop(k)
            continue
        # Transform point index to x, y, z.
        _v = np.asarray(pcd.points)[v]
        point_coordinates = {'x': _v[0], 'y': _v[1], 'z': _v[2]}
        _picked_map[k] = point_coordinates
    return _picked_map

In [3]:
# Main
WORK_DIR = '/home/commaai-03/Public/points_a'

config_file = os.path.join(WORK_DIR, 'config.json')
if not os.path.isfile(config_file):
    raise Exception('[Error]: No config.json file found. Please read help.')
    
with open(config_file, mode='r', encoding='utf-8') as f:
    config = json.load(f)
    logging.info('[Info]: Loaded config success.')
    
filenames = config['filenames']
index = filenames.index(config['index'])

for i in range(index, len(filenames)):
    _filename = os.path.join(WORK_DIR, filenames[i])
    pcd = load_pcd(_filename)
    label = label3D(pcd)
    if not label:
        continue
    labelfile = os.path.splitext(_filename)[0] + '.json'
    write_labelFile(label, labelfile)
    if not i == len(filenames) - 1:
        config['index'] = filenames[i+1]
        with open(config_file, mode='w', encoding='utf-8') as f:
            json.dump(config, f, indent=2)
            logging.info('[Info]: Index is in %s.' % config['index'])

IndexError: list index out of range